In [30]:
import numpy as np
from pathlib import Path
import shutil
import os
import glob

import spikeinterface as si
from spikeinterface.exporters import export_to_phy

import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
probe_name = 'ProbeB'

data_directories = glob.glob('/data/ecephys_*')
data_directories.sort()

session_folder = Path(data_directories[0])
sorting_folder = Path(data_directories[1])
scratch_folder = Path('/scratch')
results_folder = Path('/results')
output_folder = results_folder / probe_name

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
ecephys_compressed_folder = session_folder / 'ecephys_compressed'
sorting_curated_folder = sorting_folder / "sorting_precurated"
postprocessed_folder = sorting_folder / 'postprocessed'
ap_stream_name = f'experiment1_Record Node 104#Neuropix-PXI-100.{probe_name}-AP'
lfp_stream_name = f'experiment1_Record Node 104#Neuropix-PXI-100.{probe_name}-LFP'

In [32]:
we_recless = si.load_waveforms(postprocessed_folder / f'{ap_stream_name}_recording1', with_recording=False)

we_recless.channel_ids[:5]

/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:998: UserWarning: Versions are not the same. This might lead compatibility errors. Using spikeinterface==0.97.1 is recommended
  warnings.warn(


array(['AP1', 'AP2', 'AP3', 'AP4', 'AP5'], dtype='<U5')

In [33]:
channel_inds = np.array([int(name[2:])-1 for name in we_recless.channel_ids])

In [34]:
len(channel_inds)

369

In [35]:
we_recless.get_channel_locations()[:5]

tmp_we_path = scratch_folder / postprocessed_folder.parent.name / "postprocessed" / ap_stream_name

In [37]:
try:
    shutil.copytree(we_recless.folder, tmp_we_path)
except FileExistsError:
    print('Files already copied, ignoring')

Files already copied, ignoring


In [38]:

we_recless_local = si.load_waveforms(tmp_we_path, with_recording=False,
                                     sorting=we_recless.sorting)
print(we_recless_local)

WaveformExtractor: 369 channels - 945 units - 1 segments
  before:90 after:120 n_per_units:500 - sparse


In [39]:
phy_folder = results_folder / f"{postprocessed_folder.parent.name}_phy"


In [40]:
export_to_phy(we_recless_local, 
                   output_folder=phy_folder,
                   compute_pc_features=False,
                   remove_if_exists=True,
                   copy_binary=False)

/opt/conda/lib/python3.9/site-packages/spikeinterface/exporters/to_phy.py:85: UserWarning: Exporting to Phy with many channels and without sparsity might result in a heavy and less informative visualization. You can use use a sparse WaveformExtractor or you can use the 'sparsity' argument to enforce sparsity (see compute_sparsity())
  warnings.warn(


Run:
phy template-gui  /results/ecephys_661398_2023-03-31_17-01-09_sorted-ks2.5_phy/params.py


In [41]:
spike_locations = we_recless_local.load_extension("spike_locations").get_data()
spike_depths = spike_locations["y"]

In [42]:
# convert clusters and squeeze

clusters = np.load(phy_folder / "spike_clusters.npy")
np.save(phy_folder / "spike_clusters.npy", np.squeeze(clusters.astype('uint32')))

In [43]:
# convert times and squeeze

times = np.load(phy_folder / "spike_times.npy")
np.save(phy_folder / "spike_times.npy", np.squeeze(times / 30000.).astype('float64'))

In [44]:
# convert amplitudes and squeeze

amps = np.load(phy_folder / "amplitudes.npy")
np.save(phy_folder / "amplitudes.npy", np.squeeze(-amps / 1e6).astype('float64'))

In [46]:
np.save(phy_folder / "spike_depths.npy", spike_depths)
np.save(phy_folder / "channel_inds.npy", np.arange(len(channel_inds), dtype='int'))

In [47]:
cluster_channels = []
cluster_peakToTrough = []
cluster_waveforms = []
num_chans = []

templates = we_recless_local.get_all_templates()
channel_locs = we_recless_local.get_channel_locations()

for unit_idx, unit_id in enumerate(we_recless_local.unit_ids):
    waveform = templates[unit_idx,:,:]
    peak_channel = np.argmax(np.max(waveform, 0) - np.min(waveform,0))
    peak_waveform = waveform[:,peak_channel]
    peakToTrough = (np.argmax(peak_waveform) - np.argmin(peak_waveform)) / 30000.
    cluster_channels.append(int(channel_locs[peak_channel,1] / 10))
    cluster_peakToTrough.append(peakToTrough)
    cluster_waveforms.append(waveform)

In [48]:
np.save(phy_folder / "cluster_peakToTrough.npy", np.array(cluster_peakToTrough))
np.save(phy_folder / "cluster_waveforms.npy", np.stack(cluster_waveforms))
np.save(phy_folder / "cluster_channels.npy", np.array(cluster_channels))

In [49]:
_FILE_RENAMES = [  # file_in, file_out
    ('channel_positions.npy', 'channels.localCoordinates.npy'),
    ('channel_inds.npy', 'channels.rawInd.npy'),
    ('cluster_peakToTrough.npy', 'clusters.peakToTrough.npy'),
    ('cluster_channels.npy', 'clusters.channels.npy'),
    ('cluster_waveforms.npy', 'clusters.waveforms.npy'),
    ('spike_clusters.npy', 'spikes.clusters.npy'),
    ('amplitudes.npy', 'spikes.amps.npy'),
    ('spike_depths.npy', 'spikes.depths.npy'),
    ('spike_times.npy', 'spikes.times.npy'),
]

In [57]:
input_directory = phy_folder
output_directory = Path('/results') / probe_name

if not os.path.exists(output_directory):
    os.mkdir(output_directory)

for names in _FILE_RENAMES:
    input = input_directory / names[0]
    output = output_directory / names[1]
    shutil.copyfile(input, output)

In [51]:
qm = we_recless_local.load_extension("quality_metrics")

In [52]:
qm_data = qm.get_data()

In [53]:
we_recless_local.unit_ids

array([  0,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 103, 104, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134,
       135, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 172, 173, 174, 175,
       176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 18

In [54]:
qm_data.index.name = 'cluster_id'
qm_data['cluster_id.1'] = qm_data.index.values

In [58]:
qm_data.to_csv(output_folder / 'clusters.metrics.csv')

In [59]:
shutil.make_archive('/results/' + probe_name, 'zip', '/results/' + probe_name)

'/results/ProbeB.zip'